# Retrieve と RetrieveAndGenerate の違い

Knowledge bases for Amazon Bedrock を利用した検索には 2 種類の API が提供されています。

- [RetrieveAndGenerate](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_RetrieveAndGenerate.html)
- [Retrieve](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_Retrieve.html)

文字通り Retrieve API はベクトルデータベースから情報の取得のみを行い、 RetrieveAndGenerate API はベクトルデータベースから情報の取得した後、回答生成まで行います。

WIP

<img src="../images/01/01.png">

RetrieveAndGenerate API と Retrieve API の挙動を確認してみます。

## RetrieveAndGenerate API

In [ ]:
import boto3

KNOWLEDGEBASE_ID = "" # ナレッジベース ID を記載
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0"
knowledge_base = boto3.client("bedrock-agent-runtime")
prompt = "私はエンジニアです。オンボーディング初日に必要なことを教えてください。"

response = knowledge_base.retrieve_and_generate(
    input={"text": prompt},
    retrieveAndGenerateConfiguration={
        "type": 'KNOWLEDGE_BASE',
        "knowledgeBaseConfiguration": {
            "knowledgeBaseId": KNOWLEDGEBASE_ID,
            "modelArn": model_arn
        },
    },
)

print(response["output"]["text"])

## Retrieve API

In [ ]:
response = knowledge_base.retrieve(
    knowledgeBaseId=KNOWLEDGEBASE_ID,
    retrievalQuery={
        "text": prompt
    }
)

retrieval_results = response.get("retrievalResults")
print(retrieval_results)

このままだと、素っ気ないため Knowledge bases for Amazon Bedrock でサポートされていない Claude 3 Opus を回答文章の生成に使ってみましょう。

In [ ]:
contexts = []
model_arn = "arn:aws:bedrock:us-west-2::foundation-model/anthropic.claude-3-opus-20240229-v1:0"

for result in retrieval_results:
    result_text = result.get("content").get("text")
    contexts.append(result_text)

print(contexts_string)